 # User Experience with Artificial Intelligence 

## Background

"Machine Learning" is becoming just another tool in a developers toolbox. 

As per [StackOverflow's 2018 survey](https://insights.stackoverflow.com/survey/2018/):
* PyTorch and TensorFlow compete with React as most loved tool by 70% of devs
* 47.8% say "The developers or the people creating the AI" are primarily responsible for ML ramifications
* 72.8% say "I'm excited about the possibilities more than worried about the dangers."

* NeurIPS 2018
  * workshop on ["Ethical, Social and Governance Issues in AI"](https://nips.cc/Conferences/2018/Schedule?showEvent=10940)
  * tutorial on ["Common Pitfalls for Studying the Human Side of Machine Learning"](https://nips.cc/Conferences/2018/Schedule?showEvent=10981)
  * invited talks on [Algorithmic Bias](https://nips.cc/Conferences/2018/Schedule?showEvent=12870), [Public Policy](https://nips.cc/Conferences/2018/Schedule?showEvent=12483) and ["Human-AI Trust"](https://nips.cc/Conferences/2018/Schedule?showEvent=12301)

* Google's UX team 
  * ["Human-Centered Machine Learning"](https://medium.com/google-design/human-centered-machine-learning-a770d10562cd) to guide conversations 
  * Few great publications on [UX of AI](https://design.google/library/ux-ai/) from which I shamelessly stole the title

Machine Learning has the potential to really affect people's lives, for better or for worse, it's new and interesting.

But it's a bit like radioactivity
* **1911** Marie Curie discovers it
* **1925** your watch glows in the dark but all the painters got cancer 
* **1945** Trinity test and Hiroshima
* **1954** first nuclear power plant in Obninsk

<style>
p {
  font-size: 10px
}</style>
For Machine Learning
* **1954-1971** Few people have a neat way of building models, get excited <sup>[1](https://en.wikipedia.org/wiki/Artificial_intelligence#History), [2](https://en.wikipedia.org/wiki/Alexey_Ivakhnenko)</sup>
* **1973-1990** Machine Translation doesn't work and a famous guy 'proves' AI is bunk <sup>[3](https://en.wikipedia.org/wiki/AI_winter)</sup>


* **2012** Wait maybe not, old bug is fixed, everybody is excited again.<sup>[4](https://en.wikipedia.org/wiki/Deep_learning#Deep_learning_revolution)</sup>
* **2016** Cambridge Analytica makes a mess <sup>[5](https://en.wikipedia.org/wiki/Facebook%E2%80%93Cambridge_Analytica_data_scandal)</sup>
* **2017** Everybody gets a creepy talking speaker for their house & car 
* **2018** We hear about Cambridge Analytica but it's a bit late


## Interpretability and Explainability

David J. Spiegelhalter of the Royal Society suggests this:

"Realising the benefits of open data requires
effective communication through a more intelligent
openness: 
* data must be **accessible** and readily
located; 
* they must be **intelligible** to those who wish
to scrutinise them; 
* data must be **assessable** so that judgments can be made about their reliability and the
competence of those who created them; 
* and they must be **usable** by others. "

*-Society, Royal. "Science as an Open Enterprise." (2012).*

* Machine learning models can follow these principles.
* Users have a 'right to explanation'<sup>[1](https://en.wikipedia.org/wiki/Right_to_explanation)</sup>
* This is the law in contexts like: 
  * credit scores in the United States (ECOA)
  * general data in the EU (GDPR)

Christoph Molnar has a great book on this:
["Interpretable Machine Learning:
A Guide for Making Black Box Models Explainable"](https://christophm.github.io/interpretable-ml-book/index.html)

Next few slides:

* Interpretability vs explainability
* Implicitly interpretable models
* Review of model agnostic methods 
* Example based explainations

# Ethics in AI

Next slides:

Approaches to: 
* Fairness
* Privacy
* Trust
* Inclusion

https://twimlai.com/tag/fairness/

https://www.news.gatech.edu/2016/02/29/emergencies-should-you-trust-robot

https://design.google/library/fair-not-default/

https://github.com/dssg/aequitas 

https://pypi.org/project/aif360/

## Designing with AI in mind

Donald Norman, author of "The Design of Everyday Things" suggests designing to these principles:
* **visibility** - what you see
* **affordance** - cues for interaction
* **mapping** - connecting control and effect
* **feedback** - response to interaction
* **constraints** - what you can't do
* **consistancy** - helping users learn from patterns

Next slides: 
Applying these principles to AI

https://design.google/library/ai/

https://medium.com/google-design/human-centered-machine-learning-a770d10562cd